In [1]:
import pandas as pd
import numpy as np

In [2]:
test_file_path = './datasets/test.csv'
train_file_path = './datasets/train.csv'
test_df = pd.read_csv(test_file_path)
train_df = pd.read_csv(train_file_path)

# Remove the first column
np_train_df = train_df.to_numpy()[:, 1:]
np_test_df = test_df.to_numpy()[:, 1:]

# CONSTRAINTS
DIGIT = 0
FIRST_PIXEL = 1

In [3]:
def pos_pixel(size, i ,j):
  return i*size + j

def symmetry(image):
  size = 28
  sv = 0
  sh = 0
  
  for i in range(size):
    for j in range(int(size/2)):
      p1 = image[pos_pixel(size, i, j)]
      p2 = image[pos_pixel(size, i, (size-1)-j)]
      sv += abs(p1 - p2)
            
      p1 = image[pos_pixel(size, j, i)]
      p2 = image[pos_pixel(size, (size-1)-j, i)]
      sh += abs(p1 - p2)

  sv = sv/255
  sh = sh/255

  s = sv + sh
  
  return s
  
def intensity(image):
  return (np.sum(image)/255.0)


def get_numbers_1_and_5(df):
  filtered_data = []

  for item in df:
    if(item[0] == 1 or item[0] == 5):
      filtered_data.append(item.tolist())

  return filtered_data

def add_1_column(X):
  ones_list = map(lambda i: [1, i[0], i[1]], X)
  return np.array(list(ones_list))
  

In [4]:
new_train_df = []
images = get_numbers_1_and_5(np_train_df)

for image in images:
  symmetry_ = symmetry(image[FIRST_PIXEL:])
  intensity_ = intensity(image[FIRST_PIXEL:])

  label = -1 if image[DIGIT] == 1 else 1

  new_train_df.append([label, intensity_, symmetry_])

new_train_df = np.array(new_train_df)

In [5]:
new_test_df = []
images = get_numbers_1_and_5(np_test_df)

for image in images:
  symmetry_ = symmetry(image[FIRST_PIXEL:])
  intensity_ = intensity(image[FIRST_PIXEL:])

  label = -1 if image[DIGIT] == 1 else 1

  new_test_df.append([label, intensity_, symmetry_,])

new_test_df = np.array(new_test_df)

In [6]:
class Perceptron:
  def __init__(self, max_iter):   
    self.max_iter = max_iter         

  def fit(self, _X, _y):              #Treino
    dimension = len(_X[0])            #Dimensão do vetor de entrada
    self.w = 2 * np.random.random(size=dimension) - 1  #Pesos aleatórios
    self.best_error = dimension       #Erro inicial
    best_w = self.w                   #Melhores pesos
    
    for i in range(self.max_iter):    #Iterações

      for x_n, y_n in zip(_X, _y):    #Percorre os pontos
        y_pred = np.sign(np.dot(x_n[1:], self.w[1:]) + (x_n[0] * self.w[0])) #Calcula a saída do perceptron

        if y_pred != y_n:             #Se a saída for diferente do esperado
          self.w = self.w + x_n * y_n #Atualiza os pesos
          error = self.__error_in(zip(_X, _y)) #Calcula o erro
          if self.best_error > error: #Se o erro for menor que o melhor erro
            self.best_error = error   #Atualiza o melhor erro
            best = self.w             #Atualiza os melhores pesos
      
      self.w = best_w                 #Atualiza os pesos

  def predict(self, x_test, first_digit, second_digit):
    y_pred = np.sign(np.dot(x_test, self.w))
    return np.where(y_pred == -1.0, first_digit, second_digit)

  def get_weights(self):
    return self.w[1:]

  def get_bias(self):
    return self.w[0]

  # Private

  def __error_in(self, points):
    error = 0

    for x_n, y_n in points:
      y_pred = np.sign(np.dot(x_n, self.w))
      error += 1 if y_pred != y_n else 0

    return error

In [29]:
import numpy as np
from numpy import linalg as LA
import random

class LogisticRegression:
    def __init__(self, eta=0.1, tmax=1000, batch_size=50, lam =0):
        self.eta = eta
        self.tmax = tmax
        self.batch_size = batch_size
        self.lam = lam

    # Infere o vetor w da funçao hipotese
    #Executa a minimizao do erro de entropia cruzada pelo algoritmo gradiente de descida
    def fit(self, _X, _y):
        self.w = []
        X = np.concatenate((np.ones((len(_X),1)), _X), axis=1)
        y = np.array(_y)
        
        d = X.shape[1]
        N = X.shape[0]
        w = np.zeros(d, dtype=float)
        self.w = []
        
        for i in range(self.tmax):
            vsoma = np.zeros(d, dtype=float)

            #Escolhendo o lote de entradas
            if self.batch_size < N:
                indices = random.sample(range(N),self.batch_size)
                batchX = [X[index] for index in indices]
                batchY = [y[index] for index in indices]
            else:
                batchX = X
                batchY = y

            #computando o gradiente no ponto atual
            for xn, yn in zip(batchX, batchY):
                vsoma += (yn * xn) / (1 + np.exp((yn * w).T @ xn))
            
            gt = vsoma/self.batch_size

            if self.lam != 0:
                gt += 2*self.lam * w
            
            w = w + (self.eta*gt)

            #Condicao de parada: se ||deltaF|| < epsilon (0.0001)
            if LA.norm(gt) < 0.0001 :
                break
            w = w + (self.eta*gt)

        self.w = w
    
    # def _sigmoid(self, x):
    #     return 1 / (1 + np.exp(-(self.w[0] + self.w[1:].T @ x)))
    def _sigmoid(self, x):
        return np.exp(-(self.w[0] + self.w[1:].T @ x)) / (1 + np.exp((self.w[0] + self.w[1:].T @ x)))

    #funcao hipotese inferida pela regressa logistica  
    def predict_prob(self, X):
        pred = [self._sigmoid(x) for x in X]
        return pred

    #Predicao por classificação linear
    def predict(self, X):
        pred_classifier = [1 if self._sigmoid(x) >= 0.5 else -1 for x in X]
        return pred_classifier

    def getW(self):
        return self.w

    def getRegressionY(self, regressionX, shift=0):
        return (-self.w[0]+shift - self.w[1]*regressionX) / self.w[2]

In [7]:
def PLA_predict(X_test,w):

    preds = np.sign(np.dot(X_test,w))

    return np.where(preds == 1.0, 1, 5)

In [8]:
# PERCEPTRON

pla = Perceptron(1)
pla.fit(add_1_column(new_train_df[:, 1:]), new_train_df[:, 0])


In [9]:
pla.get_weights()

array([0.37657736, 0.31077836])

In [10]:
pla.get_bias()

0.29784607276618535

In [11]:
type(pla.w)

numpy.ndarray

In [30]:
# LOGISTIC REGRESSION

log_r = LogisticRegression()
log_r.fit(add_1_column(new_train_df[:, 1:]), new_train_df[:, 0])

/tmp/ipykernel_49021/2858402931.py:38: RuntimeWarning: overflow encountered in exp
  vsoma += (yn * xn) / (1 + np.exp((yn * w).T @ xn))


In [35]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(new_test_df[:, 0], log_r.predict(add_1_column(new_test_df))))

              precision    recall  f1-score   support

        -1.0       0.54      1.00      0.70      1528
         1.0       0.00      0.00      0.00      1324

    accuracy                           0.54      2852
   macro avg       0.27      0.50      0.35      2852
weighted avg       0.29      0.54      0.37      2852



/tmp/ipykernel_49021/2858402931.py:57: RuntimeWarning: overflow encountered in exp
  return np.exp(-(self.w[0] + self.w[1:].T @ x)) / (1 + np.exp((self.w[0] + self.w[1:].T @ x)))
/home/messias/miniconda3/envs/ufpb/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/messias/miniconda3/envs/ufpb/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/messias/miniconda3/envs/ufpb/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-scor

In [19]:
from sklearn.linear_model import LogisticRegression

log_r = LogisticRegression()
log_r.fit(new_train_df[:, 1:], new_train_df[:, 0])

LogisticRegression()

In [22]:
from sklearn.metrics import classification_report

print(classification_report(new_test_df[:, 0], log_r.predict(new_test_df[:, 1:])))

              precision    recall  f1-score   support

        -1.0       0.82      0.88      0.85      1528
         1.0       0.85      0.77      0.81      1324

    accuracy                           0.83      2852
   macro avg       0.83      0.83      0.83      2852
weighted avg       0.83      0.83      0.83      2852



In [28]:
log_r.score(new_test_df[:, 1:], new_test_df[:, 0])

0.8302945301542777

In [37]:
import matplotlib.pyplot as plt

plt.plot(log_r.predict(new_test_df[:, 1:]))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)